In [11]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

## Encoder

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, (hidden, cell)

## Attention

In [13]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.permute(0, 2, 1)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

## Decoder

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.attention = Attention(hidden_size)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        lstm_output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        attention_weights = self.attention(hidden[-1], encoder_outputs)
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        lstm_output = lstm_output.squeeze(1)
        context = context.squeeze(1)
        output = self.fc(torch.cat((lstm_output, context), dim=1))
        return output, hidden, cell, attention_weights

## Tokenization and Vocabulary Setup
First, create a vocabulary and tokenize the input sentence (e.g., "two plus four" etc).

In [15]:
# Tokenization and vocab setup
# Create a vocabulary mapping words to indices
word_to_index = {"<SOS>": 0, "<EOS>": 1, "<PAD>": 2, "two": 3, "plus": 4, "four": 5, "equals": 6, "six": 7, "three": 8, "minus": 9, "one": 10}
index_to_word = {v: k for k, v in word_to_index.items()}  # Reverse mapping for decoding

# Example input and target sequences
input_sentence = "two plus four"
target_sentence = "equals six"

# Tokenize the sentences
input_tokens = [word_to_index[word] for word in input_sentence.split()]
target_tokens = [word_to_index["<SOS>"]] + [word_to_index[word] for word in target_sentence.split()] + [word_to_index["<EOS>"]]

## Generate the math problem dataset from the tokenzied data

In [16]:
class MathWordProblemDataset(Dataset):
    def __init__(self, input_sentences, target_sentences, word_to_index):
        self.input_data = [[word_to_index[word] for word in sentence.split()] for sentence in input_sentences]
        self.target_data = [[word_to_index["<SOS>"]] + [word_to_index[word] for word in sentence.split()] + [word_to_index["<EOS>"]] for sentence in target_sentences]

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return torch.tensor(self.input_data[idx], dtype=torch.long), torch.tensor(self.target_data[idx], dtype=torch.long)

# Example data
input_sentences = ["two plus four"]
target_sentences = ["equals six"]

# Create dataset and dataloader
dataset = MathWordProblemDataset(input_sentences, target_sentences, word_to_index)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

target_length = len(target_tokens)

## Train the model

In [17]:
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the encoder and decoder
input_size = len(word_to_index)  # Total vocabulary size
output_size = len(word_to_index)  # Vocabulary size
hidden_size = 128
encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(output_size, hidden_size).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=word_to_index["<PAD>"])  # Ignore padding tokens
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

num_epochs = 100  # Define the number of epochs
for epoch in range(num_epochs):
    for input_seq, target_seq in dataloader:
        # Move data to device (CPU or GPU)
        input_seq, target_seq = input_seq.to(device), target_seq.to(device)

        # Zero the gradients
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Encoder forward pass
        encoder_outputs, (hidden, cell) = encoder(input_seq)

        # Decoder initialization
        decoder_input = torch.tensor([word_to_index["<SOS>"]]*input_seq.size(0), device=device)
        decoder_hidden, decoder_cell = hidden, cell

        # Iterate over the target sequence
        loss = 0
        for t in range(target_seq.size(1)):
            output, decoder_hidden, decoder_cell, attention_weights = decoder(
                decoder_input, decoder_hidden, decoder_cell, encoder_outputs
            )
            loss += criterion(output, target_seq[:, t])
            decoder_input = target_seq[:, t]  # Teacher forcing

        # Backpropagation and optimization
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item() / target_seq.size(1)}")


torch.save(encoder.state_dict(), "encoder.pth")
torch.save(decoder.state_dict(), "decoder.pth")

Epoch 1/100, Loss: 2.3638370037078857
Epoch 2/100, Loss: 2.1916778087615967
Epoch 3/100, Loss: 2.0262343883514404
Epoch 4/100, Loss: 1.8628535270690918
Epoch 5/100, Loss: 1.7023825645446777
Epoch 6/100, Loss: 1.5464227199554443
Epoch 7/100, Loss: 1.396775722503662
Epoch 8/100, Loss: 1.2553573846817017
Epoch 9/100, Loss: 1.1237825155258179
Epoch 10/100, Loss: 1.0029860734939575
Epoch 11/100, Loss: 0.8931413888931274
Epoch 12/100, Loss: 0.7939382791519165
Epoch 13/100, Loss: 0.7049980163574219
Epoch 14/100, Loss: 0.6261006593704224
Epoch 15/100, Loss: 0.5570806264877319
Epoch 16/100, Loss: 0.4974972605705261
Epoch 17/100, Loss: 0.44633978605270386
Epoch 18/100, Loss: 0.40208011865615845
Epoch 19/100, Loss: 0.36314263939857483
Epoch 20/100, Loss: 0.3284223973751068
Epoch 21/100, Loss: 0.29739901423454285
Epoch 22/100, Loss: 0.2698220908641815
Epoch 23/100, Loss: 0.2453511357307434
Epoch 24/100, Loss: 0.223469540476799
Epoch 25/100, Loss: 0.20362097024917603
Epoch 26/100, Loss: 0.185385450

## Test the trained model

In [18]:
# load the trained model
input_size = len(word_to_index)  # Same as during training
output_size = len(word_to_index)
hidden_size = 128  # Same as during training

encoder = Encoder(input_size, hidden_size)
decoder = Decoder(output_size, hidden_size)

# Load the trained weights
encoder.load_state_dict(torch.load("encoder.pth"))
decoder.load_state_dict(torch.load("decoder.pth"))

# Set the models to evaluation mode
encoder.eval()
decoder.eval()

# Tokenize the input sentence
new_input_sentence = "two plus four"
input_tokens = [word_to_index[word] for word in new_input_sentence.split()]
input_seq = torch.tensor(input_tokens, dtype=torch.long).unsqueeze(0)  # Add batch dimension

# generate the output sequence
# Forward pass through the encoder
encoder_outputs, (hidden, cell) = encoder(input_seq)

# Initialize the decoder
decoder_input = torch.tensor([word_to_index["<SOS>"]], dtype=torch.long)  # Start-of-Sequence token
decoder_hidden = hidden
decoder_cell = cell

# Generate output sequence
output_sequence = []
target_length = 10  # Maximum output sequence length

for _ in range(target_length):
    output, decoder_hidden, decoder_cell, _ = decoder(
        decoder_input, decoder_hidden, decoder_cell, encoder_outputs
    )
    predicted_token = output.argmax(1).item()  # Get token with the highest probability
    if predicted_token == word_to_index["<EOS>"]:  # Stop at End-of-Sequence token
        break
    output_sequence.append(predicted_token)
    decoder_input = torch.tensor([predicted_token], dtype=torch.long)

# Convert tokens back to words
output_sentence = " ".join([index_to_word[token] for token in output_sequence])
print("Output Sentence:", output_sentence)

Output Sentence: <SOS> equals six


#############################################################################################################################

# Build with padding for variable lengths

In [19]:
# Define special tokens
word_to_index = {"<SOS>": 0, "<EOS>": 1, "<PAD>": 2}  
 

# Function to tokenize a sentence and update mapping dynamically
def tokenize(sentence, word_to_index):
    tokens = []
    for word in sentence.lower().split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)  # Assign index to new words
        tokens.append(word_to_index[word])
    return tokens

# Load dataset from CSV
def load_sequences_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    return df["Problem"].tolist(), df["Solution"].tolist()

csv_file = "simple_math_problems.csv"
input_sentences, target_sentences = load_sequences_from_csv(csv_file)

# Tokenize input and target sentences
input_data = [tokenize(sentence, word_to_index) for sentence in input_sentences]
target_data = [[word_to_index["<SOS>"]] + tokenize(sentence, word_to_index) + [word_to_index["<EOS>"]]
               for sentence in target_sentences]

index_to_word = {v: k for k, v in word_to_index.items()}

# Convert tokenized sentences into tensors
input_tensors = [torch.tensor(seq) for seq in input_data]
target_tensors = [torch.tensor(seq) for seq in target_data]

# Apply dynamic padding
input_padded = pad_sequence(input_tensors, batch_first=True, padding_value=word_to_index["<PAD>"])
target_padded = pad_sequence(target_tensors, batch_first=True, padding_value=word_to_index["<PAD>"])

class MathWordProblemDataset(Dataset):
    def __init__(self, input_padded, target_padded):
        self.input_data = input_padded
        self.target_data = target_padded

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.target_data[idx]

dataset = MathWordProblemDataset(input_padded, target_padded)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = len(word_to_index)  # Vocabulary size
output_size = len(word_to_index)
hidden_size = 128

encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(output_size, hidden_size).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=word_to_index["<PAD>"])
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    for input_seq, target_seq in dataloader:
        input_seq, target_seq = input_seq.to(device), target_seq.to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, (hidden, cell) = encoder(input_seq)

        decoder_input = torch.tensor([word_to_index["<SOS>"]] * input_seq.size(0), device=device)
        decoder_hidden, decoder_cell = hidden, cell

        target_lengths = (target_seq != word_to_index["<PAD>"]).sum(dim=1)

        loss = 0
        max_target_length = target_lengths.max().item()

        for t in range(max_target_length):
            still_active = t < target_lengths
            if not still_active.any():
                break

            output, decoder_hidden, decoder_cell, _ = decoder(
                decoder_input, decoder_hidden, decoder_cell, encoder_outputs
            )
            loss += (criterion(output, target_seq[:, t]) * still_active.float()).sum() / still_active.sum()

            decoder_input = target_seq[:, t]  # Teacher forcing

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

torch.save(encoder.state_dict(), "encoder-dynamic.pth")
torch.save(decoder.state_dict(), "decoder-dynamic.pth")

Epoch 1/100, Loss: 8.916961669921875
Epoch 2/100, Loss: 2.8817436695098877
Epoch 3/100, Loss: 1.964099407196045
Epoch 4/100, Loss: 1.2228384017944336
Epoch 5/100, Loss: 3.6843483448028564
Epoch 6/100, Loss: 0.7471533417701721
Epoch 7/100, Loss: 1.1379590034484863
Epoch 8/100, Loss: 0.6730915307998657
Epoch 9/100, Loss: 4.693273067474365
Epoch 10/100, Loss: 3.0276174545288086
Epoch 11/100, Loss: 3.100637435913086
Epoch 12/100, Loss: 2.5766429901123047
Epoch 13/100, Loss: 0.18210314214229584
Epoch 14/100, Loss: 0.49282106757164
Epoch 15/100, Loss: 0.09156893938779831
Epoch 16/100, Loss: 0.12635834515094757
Epoch 17/100, Loss: 0.19743449985980988
Epoch 18/100, Loss: 2.0919671058654785
Epoch 19/100, Loss: 3.574359178543091
Epoch 20/100, Loss: 0.22900785505771637
Epoch 21/100, Loss: 2.920043468475342
Epoch 22/100, Loss: 0.3592512309551239
Epoch 23/100, Loss: 0.04478819668292999
Epoch 24/100, Loss: 0.041424792259931564
Epoch 25/100, Loss: 0.11192981153726578
Epoch 26/100, Loss: 1.62942230701

# Test the son bitch

In [1]:
# load the trained model
input_size = len(word_to_index)  # Same as during training
output_size = len(word_to_index)
hidden_size = 128  # Same as during training

encoder = Encoder(input_size, hidden_size)
decoder = Decoder(output_size, hidden_size)

# Load the trained weights
encoder.load_state_dict(torch.load("encoder-dynamic.pth"))
decoder.load_state_dict(torch.load("decoder-dynamic.pth"))

# Set the models to evaluation mode
encoder.eval()
decoder.eval()

# Tokenize the input sentence
new_input_sentence = "twelve minus fourteen" #zero point eight six
input_tokens = [word_to_index[word] for word in new_input_sentence.split()]
input_seq = torch.tensor(input_tokens, dtype=torch.long).unsqueeze(0)  # Add batch dimension


# generate the output sequence
# Forward pass through the encoder
encoder_outputs, (hidden, cell) = encoder(input_seq)

# Initialize the decoder
decoder_input = torch.tensor([word_to_index["<SOS>"]], dtype=torch.long)  # Start-of-Sequence token
decoder_hidden = hidden
decoder_cell = cell

# Generate output sequence
output_sequence = []
target_length = 100  # Maximum output sequence length

for _ in range(target_length):
    output, decoder_hidden, decoder_cell, _ = decoder(
        decoder_input, decoder_hidden, decoder_cell, encoder_outputs
    )
    predicted_token = output.argmax(1).item()  # Get token with the highest probability
    if predicted_token == word_to_index["<EOS>"]:  # Stop at End-of-Sequence token
        break
    output_sequence.append(predicted_token)
    decoder_input = torch.tensor([predicted_token], dtype=torch.long)

print(word_to_index)

print(input_tokens)

print(output_sequence)


# Convert tokens back to words
output_sentence = " ".join([index_to_word[token] for token in output_sequence])
print("Output Sentence:", output_sentence)

NameError: name 'word_to_index' is not defined